In [1]:
import argparse
import os
import re
import traceback
import pandas as pd
import importlib
from src.pipeline.heuristic_evolver import HeuristicEvolver
from src.util.util import search_file
from src.util.llm_client.get_llm_client import get_llm_client

# ----------------------
# 在下方设置参数（对应命令行参数）
# ----------------------
problem = "psp"  # 替换为具体问题名称，如 "max_cut"、"tsp" 等
seed_heuristic = "greedy_eta_assignment_74e2.py"  # 初始启发式文件路径或名称
perturbation_heuristic = "random_perturbation_50_50.py"    # local_search_perturbation_b4f8.py
smoke_test = True  # 是否进行冒烟测试
llm_config_file = os.path.join("output", "llm_config", "azure_gpt_4o.json")  # LLM配置文件路径
evolution_dir = "evolution_data"  # 进化数据集目录
validation_dir = "validation_data"  # 验证数据集目录（默认使用进化数据集目录）

perturbation_ratio = 0.1  # 扰动比例
perturbation_time = 1000  # 最大扰动尝试次数
max_refinement_round = 5  # 最大优化轮数
filter_num = 1  # 保留的顶级启发式数量
evolution_rounds = 5  # 进化迭代次数

# ----------------------
# 以下为核心执行代码，无需修改
# ----------------------
def main():
    try:
        # 搜索问题相关路径
        evolution_dir_resolved = search_file(evolution_dir, problem)
        validation_dir_resolved = search_file(validation_dir, problem)
        print('evolution_dir_resolved路径：',evolution_dir_resolved)
        print('validation_dir_resolved路径：',validation_dir_resolved)
        # 处理初始启发式文件
        if not seed_heuristic.endswith(".py"):
            seed_heuristic_file = seed_heuristic + ".py"
        else:
            seed_heuristic_file = seed_heuristic
        seed_heuristic_file = search_file(seed_heuristic_file, problem)
        if not seed_heuristic_file:
            raise FileNotFoundError(f"未找到初始启发式文件: {seed_heuristic}")
        print('seed_heuristic_file路径：',seed_heuristic_file)
        
        # 处理扰动启发式文件
        if perturbation_heuristic is None:
            # 自动查找随机扰动启发式
            heuristic_dir = os.path.join("src", "problems", problem, "heuristics", "basic_heuristics")
            if not os.path.exists(heuristic_dir):
                raise FileNotFoundError(f"启发式目录不存在: {heuristic_dir}")
            perturbation_candidates = [
                f for f in os.listdir(heuristic_dir) 
                if re.match(r"random_....\.py", f)
            ]
            if not perturbation_candidates:
                raise Exception("未找到默认扰动启发式文件（格式为 random_xxxx.py）")
            perturbation_heuristic_file = os.path.join(heuristic_dir, perturbation_candidates[0])
        else:
            if not perturbation_heuristic.endswith(".py"):
                perturbation_heuristic_file = perturbation_heuristic + ".py"
            else:
                perturbation_heuristic_file = perturbation_heuristic
            perturbation_heuristic_file = search_file(perturbation_heuristic_file, problem)
            if not perturbation_heuristic_file:
                raise FileNotFoundError(f"未找到扰动启发式文件: {perturbation_heuristic}")
        print('perturbation_heuristic_file路径：',perturbation_heuristic_file)
        # 初始化LLM客户端
        prompt_dir = os.path.join("src", "problems", "base", "prompt")
        llm_client = get_llm_client(llm_config_file, prompt_dir, None)
        
        # 初始化启发式进化器并执行进化
        heuristic_evolver = HeuristicEvolver(
            llm_client=llm_client,
            problem=problem,
            evolution_dir=evolution_dir_resolved,
            validation_dir=validation_dir_resolved
        )
        
        print("开始启发式进化...")
        evolved_heuristics = heuristic_evolver.evolve(
            basic_heuristic_file=seed_heuristic_file,
            perturbation_heuristic_file=perturbation_heuristic_file,
            perturbation_ratio=perturbation_ratio,
            perturbation_time=perturbation_time,
            max_refinement_round=max_refinement_round,
            filtered_num=filter_num,
            evolution_round=evolution_rounds,
            smoke_test=smoke_test
        )
        
        print("\n进化完成！结果如下:")
        for h in evolved_heuristics:
            print(f"启发式文件: {h[0]}, 性能分数: {h[1]}")
            
    except Exception as e:
        print(f"执行出错: {str(e)}")
        traceback.print_exc()

# 执行主函数
if __name__ == "__main__":
    main()

evolution_dir_resolved路径： output\psp\data\evolution_data
validation_dir_resolved路径： output\psp\data\validation_data
seed_heuristic_file路径： src\problems\psp\heuristics\basic_heuristics\greedy_eta_assignment_74e2.py
perturbation_heuristic_file路径： src\problems\psp\heuristics\basic_heuristics\random_perturbation_50_50.py
self.evolution_cases ['output\\psp\\data\\evolution_data\\realistic_port_01_v6_b3_t8_T12.txt', 'output\\psp\\data\\evolution_data\\realistic_port_02_v8_b4_t10_T12.txt', 'output\\psp\\data\\evolution_data\\realistic_port_03_v10_b5_t12_T12.txt', 'output\\psp\\data\\evolution_data\\realistic_port_04_v12_b5_t15_T24.txt', 'output\\psp\\data\\evolution_data\\realistic_port_05_v15_b6_t18_T24.txt', 'output\\psp\\data\\evolution_data\\realistic_port_06_v18_b7_t22_T24.txt', 'output\\psp\\data\\evolution_data\\realistic_port_07_v20_b8_t25_T24.txt', 'output\\psp\\data\\evolution_data\\realistic_port_08_v25_b10_t30_T48.txt', 'output\\psp\\data\\evolution_data\\realistic_port_09_v30_b12

KeyboardInterrupt: 

In [ ]:
evolution_dir_resolved路径： output\tsp\data\evolution_data
seed_heuristic_file路径： src\problems\tsp\heuristics\basic_heuristics\greedy_insertion_f99d.py
perturbation_heuristic_file路径： src\problems\tsp\heuristics\basic_heuristics\random_80a0.py
开始启发式进化...
iteration: 0
===============找到扰动解===============
('===============Evolution greedy_insertion_f99d on output\tsp\data\evolution_data\gr17.tsp ===============
Chat dumped to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr17.tsp\background.txt
Chat dumped to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr17.tsp\bottleneck_operations.txt
===============一共有多少个bottleneck_index=============== 3
===============第一个=============== 0
basic_heuristic_resultbasic_heuristic_resultbasic_heuristic_result========== [2195.0, 3119.0, 25529.0]
Chat dumped to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr17.tsp\suggestion_0.txt
=== 冒烟测试（smoke test）开始 ===
Chat dumped to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr17.tsp\greedy_insertion_1a93_abandoned.txt
===============一共有多少个bottleneck_index=============== 3
===============第一个=============== 1
basic_heuristic_resultbasic_heuristic_resultbasic_heuristic_result========== [2309.0, 3119.0, 25489.0]
Chat dumped to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr17.tsp\suggestion_1.txt
=== 冒烟测试（smoke test）开始 ===
=== 冒烟测试（smoke test）通过 ===
Chat dumped to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr17.tsp\greedy_insertion_0aaa.txt
Save greedy_insertion_0aaa code to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr17.tsp\greedy_insertion_0aaa.py
Chat dumped to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr17.tsp\greedy_insertion_f99d_to_greedy_insertion_0aaa.txt
========Improvement for output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr17.tsp\greedy_insertion_0aaa.py: -0.02
======================= 第五步refine开始 =======================
Chat dumped to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr17.tsp\suggestion_1_refine_0.txt
=== 冒烟测试（smoke test）开始 ===
=== 冒烟测试（smoke test）通过 ===
Chat dumped to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr17.tsp\greedy_insertion_67ea.txt
Save greedy_insertion_67ea code to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr17.tsp\greedy_insertion_67ea.py
Chat dumped to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr17.tsp\greedy_insertion_f99d_to_greedy_insertion_67ea.txt
Chat dumped to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr17.tsp\greedy_insertion_0aaa_to_greedy_insertion_67ea.txt
Improvement for output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr17.tsp\greedy_insertion_67ea.py: 0.006666666666666667
======================= 第五步refine结束 =======================
Chat dumped to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr17.tsp\suggestion_1_refine_1.txt
=== 冒烟测试（smoke test）开始 ===
=== 冒烟测试（smoke test）通过 ===
Chat dumped to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr17.tsp\greedy_insertion_ff74.txt
Save greedy_insertion_ff74 code to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr17.tsp\greedy_insertion_ff74.py
Chat dumped to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr17.tsp\greedy_insertion_0aaa_to_greedy_insertion_ff74.txt
Chat dumped to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr17.tsp\greedy_insertion_67ea_to_greedy_insertion_ff74.txt
Improvement for output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr17.tsp\greedy_insertion_ff74.py: 0.030000000000000002
======================= 第五步refine结束 =======================
===============一共有多少个bottleneck_index=============== 3
===============第一个=============== 2
basic_heuristic_resultbasic_heuristic_resultbasic_heuristic_result========== [2309.0, 3119.0, 25264.0]
Chat dumped to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr17.tsp\suggestion_2.txt
=== 冒烟测试（smoke test）开始 ===
=== 冒烟测试（smoke test）通过 ===
Chat dumped to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr17.tsp\greedy_insertion_e6a1.txt
Save greedy_insertion_e6a1 code to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr17.tsp\greedy_insertion_e6a1.py
Chat dumped to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr17.tsp\greedy_insertion_f99d_to_greedy_insertion_e6a1.txt
========Improvement for output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr17.tsp\greedy_insertion_e6a1.py: 0.02
======================= 第五步refine开始 =======================
Chat dumped to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr17.tsp\suggestion_2_refine_0.txt
=== 冒烟测试（smoke test）开始 ===
=== 冒烟测试（smoke test）通过 ===
Chat dumped to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr17.tsp\greedy_insertion_0206.txt
Save greedy_insertion_0206 code to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr17.tsp\greedy_insertion_0206.py
Chat dumped to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr17.tsp\greedy_insertion_f99d_to_greedy_insertion_0206.txt
Chat dumped to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr17.tsp\greedy_insertion_e6a1_to_greedy_insertion_0206.txt
Improvement for output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr17.tsp\greedy_insertion_0206.py: -0.0033333333333333335
======================= 第五步refine结束 =======================
Chat dumped to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr17.tsp\suggestion_2_refine_1.txt
=== 冒烟测试（smoke test）开始 ===
=== 冒烟测试（smoke test）通过 ===
Chat dumped to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr17.tsp\greedy_insertion_a216.txt
Save greedy_insertion_a216 code to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr17.tsp\greedy_insertion_a216.py
Chat dumped to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr17.tsp\greedy_insertion_e6a1_to_greedy_insertion_a216.txt
Chat dumped to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr17.tsp\greedy_insertion_0206_to_greedy_insertion_a216.txt
Improvement for output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr17.tsp\greedy_insertion_a216.py: 0.02333333333333333
======================= 第五步refine结束 =======================
iteration: 0
===============找到扰动解===============
('===============Evolution greedy_insertion_f99d on output\tsp\data\evolution_data\gr21.tsp ===============
Chat dumped to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr21.tsp\background.txt
Chat dumped to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr21.tsp\bottleneck_operations.txt
===============一共有多少个bottleneck_index=============== 4
===============第一个=============== 0
basic_heuristic_resultbasic_heuristic_resultbasic_heuristic_result========== [2157.0, 3319.0, 25525.0]
Chat dumped to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr21.tsp\suggestion_0.txt
=== 冒烟测试（smoke test）开始 ===
=== 冒烟测试（smoke test）通过 ===
Chat dumped to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr21.tsp\greedy_insertion_f489.txt
Save greedy_insertion_f489 code to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr21.tsp\greedy_insertion_f489.py
Chat dumped to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr21.tsp\greedy_insertion_f99d_to_greedy_insertion_f489.txt
========Improvement for output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr21.tsp\greedy_insertion_f489.py: 0.02333333333333333
======================= 第五步refine开始 =======================
Chat dumped to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr21.tsp\suggestion_0_refine_0.txt
=== 冒烟测试（smoke test）开始 ===
=== 冒烟测试（smoke test）通过 ===
Chat dumped to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr21.tsp\greedy_insertion_d27e.txt
Save greedy_insertion_d27e code to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr21.tsp\greedy_insertion_d27e.py
Chat dumped to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr21.tsp\greedy_insertion_f99d_to_greedy_insertion_d27e.txt
Chat dumped to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr21.tsp\greedy_insertion_f489_to_greedy_insertion_d27e.txt
Improvement for output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr21.tsp\greedy_insertion_d27e.py: -0.003333333333333336
======================= 第五步refine结束 =======================
Chat dumped to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr21.tsp\suggestion_0_refine_1.txt
=== 冒烟测试（smoke test）开始 ===
=== 冒烟测试（smoke test）通过 ===
Chat dumped to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr21.tsp\greedy_insertion_836c.txt
Save greedy_insertion_836c code to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr21.tsp\greedy_insertion_836c.py
Chat dumped to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr21.tsp\greedy_insertion_f489_to_greedy_insertion_836c.txt
Chat dumped to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr21.tsp\greedy_insertion_d27e_to_greedy_insertion_836c.txt
Improvement for output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr21.tsp\greedy_insertion_836c.py: 0.02333333333333333
======================= 第五步refine结束 =======================
===============一共有多少个bottleneck_index=============== 4
===============第一个=============== 1
basic_heuristic_resultbasic_heuristic_resultbasic_heuristic_result========== [2309.0, 3121.0, 25470.0]
Chat dumped to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr21.tsp\suggestion_1.txt
=== 冒烟测试（smoke test）开始 ===
=== 冒烟测试（smoke test）通过 ===
Chat dumped to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr21.tsp\greedy_insertion_250d.txt
Save greedy_insertion_250d code to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr21.tsp\greedy_insertion_250d.py
Chat dumped to output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr21.tsp\greedy_insertion_f99d_to_greedy_insertion_250d.txt
Traceback (most recent call last):
  File "C:\Users\gongh\HeurAgenix-main\src\pipeline\heuristic_evolver.py", line 226, in evolution_single
    suggested_improvement = sum(self.get_improvement(
                                ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\gongh\HeurAgenix-main\src\pipeline\heuristic_evolver.py", line 510, in get_improvement
    improvements = [round(env.compare(results[index], baselines[index]) / baselines[index], 2) for index in range(len(baselines))]
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\gongh\HeurAgenix-main\src\pipeline\heuristic_evolver.py", line 510, in <listcomp>
    improvements = [round(env.compare(results[index], baselines[index]) / baselines[index], 2) for index in range(len(baselines))]
                          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\gongh\HeurAgenix-main\src\problems\tsp\env.py", line 13, in <lambda>
    self.compare = lambda x, y: y - x  # 定义比较函数：若y > x则返回正值，用于判断解的优劣（成本越小越好）
                                ~~^~~
TypeError: unsupported operand type(s) for -: 'float' and 'NoneType'

iteration: 0
iteration: 10
iteration: 20
iteration: 30
iteration: 40
iteration: 50
iteration: 60
iteration: 70
iteration: 80
iteration: 90
===============找到扰动解===============
iteration: 0
===============找到扰动解===============
('===============Evolution greedy_insertion_ff74 on output\tsp\data\evolution_data\gr17.tsp ===============
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr17.tsp\background.txt
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr17.tsp\bottleneck_operations.txt
===============一共有多少个bottleneck_index=============== 2
===============第一个=============== 0
basic_heuristic_resultbasic_heuristic_resultbasic_heuristic_result========== [2159.0, 3199.0, 25407.0]
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr17.tsp\suggestion_0.txt
=== 冒烟测试（smoke test）开始 ===
=== 冒烟测试（smoke test）通过 ===
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr17.tsp\greedy_insertion_92ea.txt
Save greedy_insertion_92ea code to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr17.tsp\greedy_insertion_92ea.py
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr17.tsp\greedy_insertion_ff74_to_greedy_insertion_92ea.txt
========Improvement for output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr17.tsp\greedy_insertion_92ea.py: 0.016666666666666666
======================= 第五步refine开始 =======================
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr17.tsp\suggestion_0_refine_0.txt
=== 冒烟测试（smoke test）开始 ===
=== 冒烟测试（smoke test）通过 ===
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr17.tsp\greedy_insertion_4d4b.txt
Save greedy_insertion_4d4b code to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr17.tsp\greedy_insertion_4d4b.py
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr17.tsp\greedy_insertion_ff74_to_greedy_insertion_4d4b.txt
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr17.tsp\greedy_insertion_92ea_to_greedy_insertion_4d4b.txt
Improvement for output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr17.tsp\greedy_insertion_4d4b.py: -0.010000000000000002
======================= 第五步refine结束 =======================
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr17.tsp\suggestion_0_refine_1.txt
=== 冒烟测试（smoke test）开始 ===
=== 冒烟测试（smoke test）通过 ===
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr17.tsp\greedy_insertion_30f6.txt
Save greedy_insertion_30f6 code to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr17.tsp\greedy_insertion_30f6.py
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr17.tsp\greedy_insertion_92ea_to_greedy_insertion_30f6.txt
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr17.tsp\greedy_insertion_4d4b_to_greedy_insertion_30f6.txt
Improvement for output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr17.tsp\greedy_insertion_30f6.py: -0.016666666666666666
======================= 第五步refine结束 =======================
===============一共有多少个bottleneck_index=============== 2
===============第一个=============== 1
basic_heuristic_resultbasic_heuristic_resultbasic_heuristic_result========== [2309.0, 3040.0, 25832.0]
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr17.tsp\suggestion_1.txt
=== 冒烟测试（smoke test）开始 ===
=== 冒烟测试（smoke test）通过 ===
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr17.tsp\greedy_insertion_d339.txt
Save greedy_insertion_d339 code to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr17.tsp\greedy_insertion_d339.py
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr17.tsp\greedy_insertion_ff74_to_greedy_insertion_d339.txt
========Improvement for output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr17.tsp\greedy_insertion_d339.py: 0.01666666666666667
======================= 第五步refine开始 =======================
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr17.tsp\suggestion_1_refine_0.txt
=== 冒烟测试（smoke test）开始 ===
=== 冒烟测试（smoke test）通过 ===
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr17.tsp\greedy_insertion_298e.txt
Save greedy_insertion_298e code to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr17.tsp\greedy_insertion_298e.py
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr17.tsp\greedy_insertion_ff74_to_greedy_insertion_298e.txt
Traceback (most recent call last):
  File "C:\Users\gongh\HeurAgenix-main\src\problems\base\env.py", line 101, in run_heuristic
    operator, delta = heuristic(
                      ^^^^^^^^^^
  File "<string>", line 93, in greedy_insertion_298e
NameError: name 'avg_nearest_neighbor_dist' is not defined

Traceback (most recent call last):
  File "C:\Users\gongh\HeurAgenix-main\src\problems\base\env.py", line 101, in run_heuristic
    operator, delta = heuristic(
                      ^^^^^^^^^^
  File "<string>", line 93, in greedy_insertion_298e
NameError: name 'avg_nearest_neighbor_dist' is not defined

Traceback (most recent call last):
  File "C:\Users\gongh\HeurAgenix-main\src\problems\base\env.py", line 101, in run_heuristic
    operator, delta = heuristic(
                      ^^^^^^^^^^
  File "<string>", line 93, in greedy_insertion_298e
NameError: name 'avg_nearest_neighbor_dist' is not defined

Error and skip
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr17.tsp\suggestion_1_refine_1.txt
=== 冒烟测试（smoke test）开始 ===
=== 冒烟测试（smoke test）通过 ===
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr17.tsp\greedy_insertion_7100.txt
Save greedy_insertion_7100 code to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr17.tsp\greedy_insertion_7100.py
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr17.tsp\greedy_insertion_ff74_to_greedy_insertion_7100.txt
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr17.tsp\greedy_insertion_d339_to_greedy_insertion_7100.txt
Improvement for output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr17.tsp\greedy_insertion_7100.py: 0.02666666666666667
======================= 第五步refine结束 =======================
iteration: 0
iteration: 10
===============找到扰动解===============
('===============Evolution greedy_insertion_ff74 on output\tsp\data\evolution_data\gr21.tsp ===============
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\background.txt
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\bottleneck_operations.txt
===============一共有多少个bottleneck_index=============== 5
===============第一个=============== 0
basic_heuristic_resultbasic_heuristic_resultbasic_heuristic_result========== [2157.0, 3121.0, 25402.0]
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\suggestion_0.txt
=== 冒烟测试（smoke test）开始 ===
=== 冒烟测试（smoke test）通过 ===
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_5b9e.txt
Save greedy_insertion_5b9e code to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_5b9e.py
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_ff74_to_greedy_insertion_5b9e.txt
========Improvement for output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_5b9e.py: 0.0033333333333333335
======================= 第五步refine开始 =======================
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\suggestion_0_refine_0.txt
=== 冒烟测试（smoke test）开始 ===
=== 冒烟测试（smoke test）通过 ===
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_14d4.txt
Save greedy_insertion_14d4 code to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_14d4.py
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_ff74_to_greedy_insertion_14d4.txt
Traceback (most recent call last):
  File "C:\Users\gongh\HeurAgenix-main\src\problems\base\env.py", line 101, in run_heuristic
    operator, delta = heuristic(
                      ^^^^^^^^^^
  File "<string>", line 104, in greedy_insertion_14d4
  File "D:\AppGallery\Anaconda\Lib\site-packages\scipy\spatial\_kdtree.py", line 355, in __init__
    data = np.asarray(data)
           ^^^^^^^^^^^^^^^^
ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (17,) + inhomogeneous part.

Traceback (most recent call last):
  File "C:\Users\gongh\HeurAgenix-main\src\problems\base\env.py", line 101, in run_heuristic
    operator, delta = heuristic(
                      ^^^^^^^^^^
  File "<string>", line 104, in greedy_insertion_14d4
  File "D:\AppGallery\Anaconda\Lib\site-packages\scipy\spatial\_kdtree.py", line 355, in __init__
    data = np.asarray(data)
           ^^^^^^^^^^^^^^^^
ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (21,) + inhomogeneous part.

Traceback (most recent call last):
  File "C:\Users\gongh\HeurAgenix-main\src\problems\base\env.py", line 101, in run_heuristic
    operator, delta = heuristic(
                      ^^^^^^^^^^
  File "<string>", line 104, in greedy_insertion_14d4
  File "D:\AppGallery\Anaconda\Lib\site-packages\scipy\spatial\_kdtree.py", line 355, in __init__
    data = np.asarray(data)
           ^^^^^^^^^^^^^^^^
ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (100,) + inhomogeneous part.

Error and skip
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\suggestion_0_refine_1.txt
=== 冒烟测试（smoke test）开始 ===
=== 冒烟测试（smoke test）通过 ===
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_c56b.txt
Save greedy_insertion_c56b code to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_c56b.py
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_ff74_to_greedy_insertion_c56b.txt
Traceback (most recent call last):
  File "C:\Users\gongh\HeurAgenix-main\src\problems\base\env.py", line 101, in run_heuristic
    operator, delta = heuristic(
                      ^^^^^^^^^^
  File "<string>", line 118, in greedy_insertion_c56b
  File "D:\AppGallery\Anaconda\Lib\site-packages\scipy\spatial\_kdtree.py", line 355, in __init__
    data = np.asarray(data)
           ^^^^^^^^^^^^^^^^
ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (17,) + inhomogeneous part.

Traceback (most recent call last):
  File "C:\Users\gongh\HeurAgenix-main\src\problems\base\env.py", line 101, in run_heuristic
    operator, delta = heuristic(
                      ^^^^^^^^^^
  File "<string>", line 118, in greedy_insertion_c56b
  File "D:\AppGallery\Anaconda\Lib\site-packages\scipy\spatial\_kdtree.py", line 355, in __init__
    data = np.asarray(data)
           ^^^^^^^^^^^^^^^^
ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (21,) + inhomogeneous part.

Traceback (most recent call last):
  File "C:\Users\gongh\HeurAgenix-main\src\problems\base\env.py", line 101, in run_heuristic
    operator, delta = heuristic(
                      ^^^^^^^^^^
  File "<string>", line 118, in greedy_insertion_c56b
  File "D:\AppGallery\Anaconda\Lib\site-packages\scipy\spatial\_kdtree.py", line 355, in __init__
    data = np.asarray(data)
           ^^^^^^^^^^^^^^^^
ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (100,) + inhomogeneous part.

Error and skip
===============一共有多少个bottleneck_index=============== 5
===============第一个=============== 1
basic_heuristic_resultbasic_heuristic_resultbasic_heuristic_result========== [2195.0, 3199.0, 25462.0]
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\suggestion_1.txt
=== 冒烟测试（smoke test）开始 ===
=== 冒烟测试（smoke test）通过 ===
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_bc7f.txt
Save greedy_insertion_bc7f code to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_bc7f.py
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_ff74_to_greedy_insertion_bc7f.txt
========Improvement for output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_bc7f.py: 0.01
======================= 第五步refine开始 =======================
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\suggestion_1_refine_0.txt
=== 冒烟测试（smoke test）开始 ===
=== 冒烟测试（smoke test）通过 ===
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_bcd6.txt
Save greedy_insertion_bcd6 code to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_bcd6.py
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_ff74_to_greedy_insertion_bcd6.txt
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_bc7f_to_greedy_insertion_bcd6.txt
Improvement for output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_bcd6.py: -0.010000000000000002
======================= 第五步refine结束 =======================
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\suggestion_1_refine_1.txt
=== 冒烟测试（smoke test）开始 ===
=== 冒烟测试（smoke test）通过 ===
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_eef0.txt
Save greedy_insertion_eef0 code to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_eef0.py
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_bc7f_to_greedy_insertion_eef0.txt
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_bcd6_to_greedy_insertion_eef0.txt
Improvement for output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_eef0.py: -0.013333333333333336
======================= 第五步refine结束 =======================
===============一共有多少个bottleneck_index=============== 5
===============第一个=============== 2
basic_heuristic_resultbasic_heuristic_resultbasic_heuristic_result========== [2157.0, 3121.0, 25420.0]
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\suggestion_2.txt
=== 冒烟测试（smoke test）开始 ===
=== 冒烟测试（smoke test）通过 ===
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_52ed.txt
Save greedy_insertion_52ed code to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_52ed.py
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_ff74_to_greedy_insertion_52ed.txt
========Improvement for output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_52ed.py: -0.006666666666666667
======================= 第五步refine开始 =======================
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\suggestion_2_refine_0.txt
=== 冒烟测试（smoke test）开始 ===
=== 冒烟测试（smoke test）通过 ===
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_61b3.txt
Save greedy_insertion_61b3 code to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_61b3.py
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_ff74_to_greedy_insertion_61b3.txt
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_52ed_to_greedy_insertion_61b3.txt
Improvement for output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_61b3.py: -0.043333333333333335
======================= 第五步refine结束 =======================
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\suggestion_2_refine_1.txt
=== 冒烟测试（smoke test）开始 ===
=== 冒烟测试（smoke test）通过 ===
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_c123.txt
Save greedy_insertion_c123 code to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_c123.py
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_52ed_to_greedy_insertion_c123.txt
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_61b3_to_greedy_insertion_c123.txt
Improvement for output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_c123.py: -0.02
======================= 第五步refine结束 =======================
===============一共有多少个bottleneck_index=============== 5
===============第一个=============== 3
basic_heuristic_resultbasic_heuristic_resultbasic_heuristic_result========== [2247.0, 3040.0, 25420.0]
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\suggestion_3.txt
=== 冒烟测试（smoke test）开始 ===
=== 冒烟测试（smoke test）通过 ===
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_ed7a.txt
Save greedy_insertion_ed7a code to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_ed7a.py
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_ff74_to_greedy_insertion_ed7a.txt
========Improvement for output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_ed7a.py: 5.782411586589357e-19
======================= 第五步refine开始 =======================
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\suggestion_3_refine_0.txt
=== 冒烟测试（smoke test）开始 ===
=== 冒烟测试（smoke test）通过 ===
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_7df6.txt
Save greedy_insertion_7df6 code to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_7df6.py
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_ff74_to_greedy_insertion_7df6.txt
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_ed7a_to_greedy_insertion_7df6.txt
Improvement for output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_7df6.py: -0.016666666666666666
======================= 第五步refine结束 =======================
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\suggestion_3_refine_1.txt
=== 冒烟测试（smoke test）开始 ===
=== 冒烟测试（smoke test）通过 ===
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_9316.txt
Save greedy_insertion_9316 code to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_9316.py
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_ed7a_to_greedy_insertion_9316.txt
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_7df6_to_greedy_insertion_9316.txt
Improvement for output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_9316.py: -0.043333333333333335
======================= 第五步refine结束 =======================
===============一共有多少个bottleneck_index=============== 5
===============第一个=============== 4
basic_heuristic_resultbasic_heuristic_resultbasic_heuristic_result========== [2309.0, 3040.0, 25481.0]
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\suggestion_4.txt
=== 冒烟测试（smoke test）开始 ===
=== 冒烟测试（smoke test）通过 ===
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_c019.txt
Save greedy_insertion_c019 code to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_c019.py
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_ff74_to_greedy_insertion_c019.txt
========Improvement for output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_c019.py: -0.0033333333333333335
======================= 第五步refine开始 =======================
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\suggestion_4_refine_0.txt
=== 冒烟测试（smoke test）开始 ===
=== 冒烟测试（smoke test）通过 ===
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_881c.txt
Save greedy_insertion_881c code to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_881c.py
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_ff74_to_greedy_insertion_881c.txt
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_c019_to_greedy_insertion_881c.txt
Improvement for output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_881c.py: -0.02666666666666666
======================= 第五步refine结束 =======================
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\suggestion_4_refine_1.txt
=== 冒烟测试（smoke test）开始 ===
=== 冒烟测试（smoke test）通过 ===
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_9897.txt
Save greedy_insertion_9897 code to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_9897.py
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_c019_to_greedy_insertion_9897.txt
Chat dumped to output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_881c_to_greedy_insertion_9897.txt
Improvement for output\tsp\evolution_result\greedy_insertion_ff74.evolution\gr21.tsp\greedy_insertion_9897.py: 0.023333333333333334
======================= 第五步refine结束 =======================
iteration: 0
iteration: 10
iteration: 20
iteration: 30
iteration: 40
iteration: 50
iteration: 60
iteration: 70
iteration: 80
iteration: 90
===============找到扰动解===============

进化完成！结果如下:
启发式文件: output\tsp\evolution_result\greedy_insertion_f99d.evolution\gr17.tsp\greedy_insertion_ff74.py, 性能分数: 0.030000000000000002